---
title: "CTE (Common Table Expression) improves code readability and reduces repetition"
format:
  html:
    toc: true
execute:
    eval: false
    output: true
---

## Why use a CTE

A CTE is a named select statement that can be reused in a single query.

Complex SQL queries often involve multiple subqueries. Multiple sub-queries make the code hard to read. Use a Common Table Expression (CTE) to make your queries readable.

CTEs also make testing complex queries simpler.

## How to define a CTE 

Use the `WITH` keyword to start defining a CTE; the WITH keyword is not necessary for consecutive CTE definitions.

In [ ]:
%%sql
use prod.db

In [ ]:
%%sql
-- CTE definition
WITH
  supplier_nation_metrics AS ( -- CTE 1 defined using WITH keyword
    SELECT
      n.n_nationkey,
      SUM(l.l_QUANTITY) AS num_supplied_parts
    FROM
      lineitem l
      JOIN supplier s ON l.l_suppkey = s.s_suppkey
      JOIN nation n ON s.s_nationkey = n.n_nationkey
    GROUP BY
      n.n_nationkey
  ),
  buyer_nation_metrics AS ( -- CTE 2 defined just as a name
    SELECT
      n.n_nationkey,
      SUM(l.l_QUANTITY) AS num_purchased_parts
    FROM
      lineitem l
      JOIN orders o ON l.l_orderkey = o.o_orderkey
      JOIN customer c ON o.o_custkey = c.c_custkey
      JOIN nation n ON c.c_nationkey = n.n_nationkey
    GROUP BY
      n.n_nationkey
  )
SELECT -- The final select will not have a comma before it
  n.n_name AS nation_name,
  s.num_supplied_parts,
  b.num_purchased_parts
FROM
  nation n
  LEFT JOIN supplier_nation_metrics s ON n.n_nationkey = s.n_nationkey
  LEFT JOIN buyer_nation_metrics b ON n.n_nationkey = b.n_nationkey
LIMIT 10;

Note that the last CTE does not have a `,` after it. 

Let's look at another example: Calculate the money lost due to discounts. Use the lineitem to retrieve the prices of items (excluding discounts) that are part of an order and compare them to the order.

The `l_extendedprice` column does not include discounts & the `o_totalprice` column includes discounts.

Hint: 
Figure out the grain in which the comparison needs to be made. 
Think in steps: get the price of all items in an order without discounts, and then compare it to the order's data, whose total price has been computed with discounts.


In [ ]:
%%sql
WITH lineitem_agg AS (
    SELECT 
        l_orderkey,
        SUM(l_extendedprice) AS total_price_without_discount
    FROM 
        lineitem
    GROUP BY 
        l_orderkey
)
SELECT 
    o.o_orderkey,
    o.o_totalprice, 
    l.total_price_without_discount - o.o_totalprice AS amount_lost_to_discount
FROM 
    orders o
JOIN 
    lineitem_agg l ON o.o_orderkey = l.l_orderkey
ORDER BY 
    o.o_orderkey;

Here are the schemas of orders and lineitem tables.

![TPCH Data Model](./images/tpch_erd.png)

## Recreating similar CTE is a sign that it should be a table

A sql query with multiple temporary tables is better than a 1000-line SQL query with numerous CTEs.

Keep the number of CTEs per query small (depends on the size of the query, but typically < 5)

Assume that you have stakeholders running the below query multiple times as needed. 

When multiple stakeholders repeatedly run the exact CTE definition, it is usually an indication that the CTE should be created as a table or view to ensure stakeholders have a unified definition.

```sql
WITH orders AS (
    SELECT
        order_id AS o_order_id,
        customer_id AS o_customer_id,
        order_status AS o_order_status,
        CAST(order_purchase_timestamp AS TIMESTAMP) AS o_order_purchase_timestamp,
        CAST(order_approved_at AS TIMESTAMP) AS o_order_approved_at,
        CAST(order_delivered_carrier_date AS TIMESTAMP) AS o_order_delivered_carrier_date,
        CAST(order_delivered_customer_date AS TIMESTAMP) AS o_order_delivered_customer_date,
        CAST(order_estimated_delivery_date AS TIMESTAMP) AS o_order_estimated_delivery_date
    FROM raw_layer.orders
),
stg_customers AS (
    SELECT
        customer_id AS sc_customer_id,
        zipcode AS sc_zipcode,
        city AS sc_city,
        state_code AS sc_state_code,
        CAST(datetime_created AS TIMESTAMP) AS sc_datetime_created,
        CAST(datetime_updated AS TIMESTAMP) AS sc_datetime_updated,
        dbt_valid_from AS sc_dbt_valid_from,
        dbt_valid_to AS sc_dbt_valid_to
    FROM customer_snapshot
),
state AS (
    SELECT
        CAST(state_id AS INT) AS s_state_id,
        CAST(state_code AS STRING) AS s_state_code,
        CAST(state_name AS STRING) AS s_state_name
    FROM raw_layer.state
),
dim_customers AS (
    SELECT
        c.sc_customer_id AS dc_customer_id,
        c.sc_zipcode AS dc_zipcode,
        c.sc_city AS dc_city,
        c.sc_state_code AS dc_state_code,
        s.s_state_name AS dc_state_name,
        c.sc_datetime_created AS dc_datetime_created,
        c.sc_datetime_updated AS dc_datetime_updated,
        CAST(c.sc_dbt_valid_from AS TIMESTAMP) AS dc_valid_from,
        CASE
            WHEN c.sc_dbt_valid_to IS NULL THEN CAST('9999-12-31' AS TIMESTAMP)
            ELSE CAST(c.sc_dbt_valid_to AS TIMESTAMP)
        END AS dc_valid_to
    FROM stg_customers AS c
    INNER JOIN state AS s ON c.sc_state_code = s.s_state_code
)
SELECT
    o.o_order_id,
    o.o_customer_id,
    o.o_order_status,
    o.o_order_purchase_timestamp,
    o.o_order_approved_at,
    o.o_order_delivered_carrier_date,
    o.o_order_delivered_customer_date,
    o.o_order_estimated_delivery_date,
    c.dc_zipcode AS customer_zipcode,
    c.dc_city AS customer_city,
    c.dc_state_code AS customer_state_code,
    c.dc_state_name AS customer_state_name
FROM orders AS o
INNER JOIN dim_customers AS c ON
    o.o_customer_id = c.dc_customer_id
    AND o.o_order_purchase_timestamp >= c.dc_valid_from
    AND o.o_order_purchase_timestamp <= c.dc_valid_to;
```

## Exercises

1. Sellers who sell at least one of the top 10 selling parts. Topics: CTE

## Recommended reading

1. https://www.startdataengineering.com/post/using-common-table-expression-in-redshift/ 